## Setup

Let’s start by checking to see what GPU we’ve been assigned. Ideally we get a V100, but a P100 is fine too. Other GPUs may lead to issues.

In [6]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-8b631b5f-9f69-0a5a-1744-5552301006db)


Next let’s connect our Google Drive account. This is optional but highly recommended.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Install repo

The next cell will install the StyleGAN repository in Google Drive. If you have already installed it it will just move into that folder. If you don’t have Google Drive connected it will just install the necessary code in Colab.

In [8]:
import os
!pip install gdown --upgrade

if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14951 sha256=eafef50040ab416a06d4e18979c1618ecc80767dc909986bc01cffdd67ea5805
  Stored in directory: /root/.cache/pip/wheels/3d/ec/b0/a96d1d126183f98570a785e6bf8789fca559853a9260e928e1
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch


In [4]:
#Uninstall new JAX
!pip uninstall jax jaxlib -y
#GPU frontend
!pip install "jax[cuda11_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
#CPU frontend
#!pip install jax[cpu]==0.3.10
#Downgrade Pytorch
!pip uninstall torch torchvision -y
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install timm==0.4.12 ftfy==6.1.1 ninja==1.10.2

Found existing installation: jax 0.3.14
Uninstalling jax-0.3.14:
  Successfully uninstalled jax-0.3.14
Found existing installation: jaxlib 0.3.14+cuda11.cudnn805
Uninstalling jaxlib-0.3.14+cuda11.cudnn805:
  Successfully uninstalled jaxlib-0.3.14+cuda11.cudnn805
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     |████████████████████████████████| 939 kB 13.7 MB/s 
     |████████████████████████████████| 175.7 MB 5.4 kB/s 
  Created wheel for jax: filename=jax-0.3.10-py3-none-any.whl size=1088066 sha256=f82039839c7f7ba644d36597c308e9ff74c6d0328b0a3321a467b78346f2ec0d
  Stored in directory: /root/.cache/pip/wheels/1c/05/23/36730377cd7311156f1e5eb5e7c683d5cdac1654658c095cc5
Successfully built jax
Found existing installation: torch 1.12.1+cu113
Uninstalling torch-1.12.1+cu113:
  Successfully uninstalled torch-1.12.1+cu113
Found existing installation: tor

You probably don’t need to run this, but this will update your repo to the latest and greatest.

In [ ]:
# %cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
# !git config --global user.name "test"
# !git config --global user.email "test@test.com"
# !git fetch origin
# !git pull
# !git stash
# !git checkout origin/main -- train.py generate.py legacy.py closed_form_factorization.py flesh_digression.py apply_factor.py README.md calc_metrics.py training/stylegan2_multi.py training/training_loop.py util/utilgan.py

## Dataset Preparation

Upload a .zip of square images to the `datasets` folder. Previously you had to convert your model to .tfrecords. That’s no longer needed :)

## Train model

Below are a series of variables you need to set to run the training. You probably won’t need to touch most of them.

* `dataset_path`: this is the path to your .zip file
* `resume_from`: if you’re starting a new dataset I recommend `'ffhq1024'` or `'./pretrained/wikiart.pkl'`
* `mirror_x` and `mirror_y`: Allow the dataset to use horizontal or vertical mirroring.

In [4]:
#required: definitely edit these!
dataset_path = '/content/drive/MyDrive/Colab Notebooks/output_images/train_set.zip'
resume_from = '/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl'
aug_strength = 0.0
train_count = 0
mirror_x = True
#mirror_y = False

#optional: you might not need to edit these
gamma_value = 50.0
augs = 'bg'
config = '11gb-gpu'
snapshot_count = 4

In [5]:
!python train.py --gpus=1 --cfg=$config --metrics=None --outdir=./results --data=$dataset_path --snap=$snapshot_count --resume=$resume_from --augpipe=$augs --initstrength=$aug_strength --gamma=$gamma_value --mirror=$mirror_x --mirrory=False --nkimg=$train_count

python3: can't open file 'train.py': [Errno 2] No such file or directory


### Resume Training

Once Colab has shutdown, you’ll need to resume your training. Reset the variables above, particularly the `resume_from` and `aug_strength` settings.

1. Point `resume_from` to the last .pkl you trained (you’ll find these in the `results` folder)
2. Update `aug_strength` to match the augment value of the last pkl file. Often you’ll see this in the console, but you may need to look at the `log.txt`. Updating this makes sure training stays as stable as possible.
3. You may want to update `train_count` to keep track of your training progress.

Once all of this has been reset, run that variable cell and the training command cell after it.

## Convert Legacy Model

If you have an older version of a model (Tensorflow based StyleGAN, or Runway downloaded .pkl file) you’ll need to convert to the newest version. If you’ve trained in this notebook you do **not** need to use this cell.

`--source`: path to model that you want to convert

`--dest`: path and file name to convert to.

In [ ]:
!python legacy.py --source=/content/drive/MyDrive/runway.pkl --dest=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/runway.pkl

## Testing/Inference

Also known as "Inference", "Evaluation" or "Testing" the model. This is the process of usinng your trained model to generate new material, usually images or videos.

### Generate Single Images

`--network`: Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`: This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation`: Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. The smaller the number the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it's infinite. 


In [ ]:
!python generate.py --outdir=/content/out/images/ --trunc=0.8 --seeds=0 --network=/content/drive/MyDrive/ladiescrop-network-snapshot-012885.pkl

#### Non-Square outputs

We can modify the model to output images that are not square. This isn’t as good as training a rectangular model, but with the right model it can still look nice.

* `--size` size takes in a value of `xdim-ydim`. For example, to generate a 1920x1080 image use `1920-1080`
* `--scale-type` This determines the padding style to apply in the additional space. There are four options: `pad`, `padside`, `symm`, and `symmside`. I recommend trying each one to see what works best with your images.


In [ ]:
!python generate.py --outdir=/content/out/images/ --trunc=0.7 --size=1820-1024 --scale-type=symm --seeds=0-499 --network=/content/crystal.pkl

We can use these options for any image or video generation commands (excluding projection).

### Truncation Traversal

Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

#### Options 
`--network`: Again, this should be the path to your .pkl file.

`--seeds`: Pass this only one seed. Pick a favorite from your generated images.

`--start`: Starting truncation value.

`--stop`: Stopping truncation value. This should be larger than the start value. (Will probably break if its not).

`--increment`: How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)


In [ ]:
!python generate.py --process="truncation" --outdir=/content/out/trunc-trav-3/ --start=-0.8 --stop=2.8 --increment=0.02 --seeds=470 --network=/content/drive/MyDrive/stylegan2-transfer-models/mixed6k-network-snapshot-016470.pkl

### Interpolations

Interpolation is the process of generating very small changes to a vector in order to make it appear animated from frame to frame.

We’ll look at different examples of interpolation below.

#### Options

`--network`: path to your .pkl file

`--interpolation`: Walk type defines the type of interpolation you want. In some cases it can also specify whether you want the z space or the w space.

`--frames`: How many frames you want to produce. Use this to manage the length of your video.

`--trunc`: truncation value

#### Linear Interpolation

In [ ]:
!python generate.py --outdir=/content/out/video1-w-0.5/ --space="z" --trunc=0.5 --process="interpolation" --seeds=463,470 --network=/content/drive/MyDrive/stylegan2-transfer-models/mixed6k-network-snapshot-016470.pkl

In [ ]:
!python generate.py --outdir=out/video1-w/ --space="w" --trunc=1 --process="interpolation" --seeds=85,265,297,849 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

#### Slerp Interpolation

This gets a little heady, but technically linear interpolations are not the best in high-dimensional GANs. [This github link](https://github.com/soumith/dcgan.torch/issues/14) is one of the more popular explanations ad discussions.

In reality I do not find a huge difference between linear and spherical interpolations (the difference in z- and w-space is enough in many cases), but I’ve implemented slerp here for anyone interested.

In [ ]:
!python generate.py --outdir=out/slerp-z/ --space="z" --trunc=1 --process="interpolation" --interpolation="slerp" --seeds=85,265,297,849 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl --frames=24

In [ ]:
!python generate.py --outdir=out/slerp-w/ --space="w" --trunc=1 --process="interpolation" --interpolation="slerp" --seeds=85,265,297,849 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl --frames=12

#### Noise Loop

If you want to just make a random but fun interpolation of your model the noise loop is the way to go. It creates a random path thru the z space to show you a diverse set of images.

`--interpolation="noiseloop"`: set this to use the noise loop funtion

`--diameter`: This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `--frames` can help determine how fast the video feels.

`--random_seed`: this allows you to change your starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point (and if you want to return to it you can use the same seed multiple times).

Noise loops currently only work in z space.

In [ ]:
!python generate.py --outdir=out/video-noiseloop-0.9d/ --trunc=0.8 --process="interpolation" --interpolation="noiseloop" --diameter=0.9 --random_seed=100 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

#### Circular Loop

The noise loop is, well, noisy. This circular loop will feel much more even, while still providing a random loop.

I recommend using a higher `--diameter` value than you do with noise loops. Something between `50.0` and `500.0` alongside `--frames` can help control speed and diversity.

In [ ]:
!python generate.py --outdir=out/video-circularloop/ --trunc=1 --process="interpolation" --interpolation="circularloop" --diameter=800.00 --frames=720 --random_seed=90 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

## Projection

### Basic Projector

*   `--target`: this is a path to the image file that you want to "find" in your model. This image must be the exact same size as your model.
*   `--num-steps`: how many iterations the projctor should run for. Lower will mean less steps and less likelihood of a good projection. Higher will take longer but will likely produce better images.



In [ ]:
!python projector.py --help

In [ ]:
!python projector.py --network=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00023-chin-morris-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000304.pkl --outdir=/content/projector/ --target=/content/img005421_0.png --num-steps=200 --seed=0

### Peter Baylies’ Projector

In [ ]:
!python /content/stylegan2-ada-pytorch/pbaylies_projector.py --help

In [ ]:
!python /content/stylegan2-ada-pytorch/pbaylies_projector.py --network=/content/ladiesblack.pkl --outdir=/content/projector-no-clip-006265-4-inv-3k/ --target-image=/content/img006265-4-inv.png --num-steps=3000 --use-clip=False --use-center=False --seed=99

## Combine NPZ files together

In [ ]:
!python combine_npz.py --outdir=/content/npz --npzs='/content/projector-no-clip-006264-1-inv-3k/projector-no-clip-006264-1-inv-3k.npz,/content/projector-no-clip-006265-1-inv-3k/projector-no-clip-006265-1-inv-3k.npz,/content/projector-no-clip-006264-5-inv-3k/projector-no-clip-006264-5-inv-3k.npz,/content/projector-no-clip-006265-3-inv-3k/projector-no-clip-006265-3-inv-3k.npz,/content/projector-no-clip-006265-4-inv-3k/projector-no-clip-006265-4-inv-3k.npz,/content/projector-no-clip-006264-1-inv-3k/projector-no-clip-006264-1-inv-3k.npz'

In [ ]:
!python generate.py --help

In [ ]:
!python generate.py --process=interpolation --interpolation=linear --easing=easeInOutQuad --space=w --network=/content/ladiesblack.pkl --outdir=/content/combined-proj/ --projected-w=/content/npz/combined.npz --frames=120

## Feature Extraction using Closed Form Factorization

Feature Extraction is the process of finding “human readable” vectors in a StyleGAN model. For example, let’s say you wanted to find a vector that could open or close a mouth in a face model.

The feature extractor tries to automate the procss of finding important vectors in your model.

`--ckpt`: This is the path to your .pkl file. In other places its called `--network` (It’s a long story for why its name changed here)
`--out`: path to save your output feature vector file. The file name must end in `.pt`!

In [ ]:
!python closed_form_factorization.py --out=/content/ladiesblack-cff.pt --ckpt=/content/ladiesblack.pkl

Once this cell is finished you’ll want to save that `.pt` file somewhere for reuse.

This process just created the vctor values, but we need to test it on some seed values to determine what each vector actually changes. The `apply_factor.py` script does this.

Arguments to try:


*   `-i`: This stands for index. By default, the cell above will produce 512 vectors, so `-i` can be any value from 0 to 511. I recommend starting with a higher value.
*   `-d`: This stands for degrees. This means how much change you want to see along th vector. I recommend a value between 5 and 10 to start with.
*   `--seeds`: You know what these are by now right? :)
*   `--ckpt`: path to your .pkl file
*   `--video`: adding this to your argument will produce a video that animates your seeds along the vector path. I find it much easier to figure out what’s changing with an animation.
*   `--output`: where to save the images/video
*   `--space`: By default this will use the w space to reduce entanglement

Lastly you need to add the path to the `.pt` file you made in th above cell. It’s weird, but you don’t need to add any arguments bfore it, just make sure its after `apply_factor.pt`



In [ ]:
!python apply_factor.py -i 0 -d 10 --seeds 5,10 --ckpt /content/ladiesblack.pkl /content/ladiesblack-cff.pt --output /content/cff-vid/ --video

That just produced images or video for a single vector, but there are 511 more! To generate every vector, you can uuse the cell below. Update any arguments you want, but don’t touch the `-i {i}` part.

**Warning:** This takes a long time, especially if you have more than one seed value (pro tip: don’t usee more than one seed value)! Also, this will take up a good amount of space in Google Drive. You’ve been warned!

In [ ]:
for i in range(512):
  !python apply_factor.py -i {i} -d 10 --seeds 177 --ckpt /content/drive/MyDrive/network-snapshot-008720.pkl /content/ladies-black-cff.pt --output /content/drive/MyDrive/ladiesblack-cff-17/ --video #--out_prefix 'ladiesblack-factor-{i}'

# Layer Manipulations

The following scripts allow you to modify various resolution layers of the StyleGAN model.

## Flesh Digressions

Flesh Digressions works by manipulating the vectors in the base 4x4 layer. By doing this while leaving all the other layers untouched you can create a warping and twisting version of images from your model.

In [ ]:
!python flesh_digression.py --pkl /content/stylegan2-ada-pytorch/pretrained/wikiart.pkl --psi 0.5 --seed 9999

## Network Blending
You can take two completely different models and combine them by splitting them at a specific resolution and combining the lower layers of one model and the higher layers of another.

(Note: this tends to work best when one of the models is transfer learned from the other)

In [ ]:
!python blend_models.py --lower_res_pkl /content/ffhq-pt.pkl --split_res 64 --higher_res_pkl /content/bone-bone-pt.pkl --output_path /content/ffhq-bonebone-split64.pkl

You can now take the output .pkl file and use that with any of the generation tools above.